In [1]:
import pandas as pd
import numpy as np
import os
import ast
import matplotlib.pyplot as plt
import requests
import json
import prettytable
path = os.getcwd()

# Objective

Analyze housing prices and labor trends across the US and when partitioned by individual states. Calculate both year-over-year and cumulative inflation and adjust prices according to cumulative inflation with 2012 as a base year. Calculate the increases in both annual median salaries and median single-family home sale prices when adjusting for inflation to isolate price increases not due to inflation. Find the states with the lowest median housing prices and highest median salaries to create actionable recommendations for determining which states are best for purchasing homes and work.

## Data Provenance
1. Redfin Housing Data - provides median home sale prices across 30 day monitoring periods
2. Bureau of Labor Statistics (BLS) Labor Data - provides wage data by area and occupation
3. Bureau of Labor Statistics (BLS) CPI Data - provides CPI data that can be used to calculate inflation

## Key Analysis Points
1. National: analysis of US Salary and single-family home sales on national level

    a. How have housing prices changed over time?
    
    b. What happens if we adjust for inflation?
    
    c. How has the quantity of houses sold in the market changed over time?
    
    d. How has salary changed over time in respect to housing prices?
    
2. State-level: Analysis of single-familng home sales per state

    a. Which states are the most expensive/cheap to buy housing in?
    
    b. Which states have the lowest housing prices and highest salaries?
    
    c. Which states are the most affordable to buy a house in assuming the individual also works in that state?
    
    d. Does it make sense to adjust recent prices for inflation when in consideration of decision-making actionability?

In [2]:
# Reading in BLS labor data
def read_wage(start_year, end_year):
    # Can specify custom years so long as all files exist in bucket
    end_str = str(end_year)
    labor_bucket = path + '\\Desktop\\housing_data\\wage\\'    
    df_dict = {}
    while start_year <= end_year:
        start_str = str(start_year)
        base_file = labor_bucket + f'state_M{start_str}_dl.xlsx'
        key = f'df_{start_str}'
        
        # Adding year col, standardizing each col header
        df_dict[key] = pd.read_excel(base_file, sheet_name = 0)
        df_dict[key].insert(0, 'YEAR', key[-4:])
        df_dict[key].columns = [col.upper() for col in df_dict[key].columns]
        start_year += 1
    
    return df_dict

# Need a way to map state to the merged df, BLS col headers differ between datasets
def get_state(df):
    state_to_code = dict(set(zip(df['AREA_TITLE'], df['AREA'])))
    code_to_state = dict(set(zip(df['AREA'], df['AREA_TITLE'])))
    return state_to_code, code_to_state

def merge_dfs(df_dict):
    for key in df_dict:
        df_dict[key] = df_dict[key][['YEAR', 'AREA', 'OCC_CODE', 'A_MEDIAN']]
        # Focus on overall aggregate of occupations       
        df_dict[key] = df_dict[key][df_dict[key]['OCC_CODE'] == '00-0000']
    # Vstack dfs so long as same cols
    df = pd.concat(df_dict.values(), axis = 0)
    
    return df

In [3]:
df_dict = read_wage(2012, 2023)
state_to_code, code_to_state = get_state(df_dict['df_2023'])
wages = merge_dfs(df_dict)
wages['STATE'] = wages['AREA'].map(code_to_state)
wages['STATE'].isnull().sum()

0

In [4]:
state_path = path + '\\Desktop\\housing_data\\state_market_tracker.tsv000'

state = pd.read_csv(state_path, sep = '\t')
state = state[state['property_type'] == 'Single Family Residential']

## Constructing curated dataset

1. Year: the year that the listing ended (using end year to preserve recency of timeframe)
2. State: State of listing, can be used for aggregations
3. Property type: Which property types are increasing faster in value?
4. Median Sale Price grouped by year

In [5]:
# Create year col
state['year'] = state['period_end'].str[:4]

df = state[['year', 'state', 'state_code', 'property_type', 'median_sale_price', 'homes_sold']]

# Create df with median sale per state per year 
grouped_year_state = df.groupby(['year', 'state', 'state_code']).agg({'median_sale_price': 'median', 
                                                        'homes_sold': 'sum'}).reset_index()
#grouped_year_state['median_sale_price'] = grouped_year_state['median_sale_price'].astype('Int64')
grouped_year = df.groupby(['year']).agg({'median_sale_price': 'median', 'homes_sold': 'sum'}).reset_index()
grouped_year['median_sale_price'] = grouped_year['median_sale_price'].astype('Int64')

In [6]:
# This just a modified copy of the sample code from https://www.bls.gov/developers/api_python.htm#python1 so that it returns df
def get_cpi(start, end):
    headers = {'Content-type': 'application/json'}
    data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],'startyear':start, 'endyear':end})
    p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    
    data_list = []
    
    for series in json_data['Results']['series']:
        seriesId = series['seriesID']
        for item in series['data']:
            year = item['year']
            period = item['period']
            value = item['value']
            footnotes = ""
            for footnote in item['footnotes']:
                if footnote:
                    footnotes += footnote['text'] + ','

            if 'M01' <= period <= 'M12':
                data_list.append([seriesId, year, period, value, footnotes[0:-1]])

    df = pd.DataFrame(data_list, columns=["series id", "year", "period", "value", "footnotes"])
    
    return df

## Calculating Inflation

In [7]:
cpi_1 = get_cpi(2012, 2020)
cpi_2 = get_cpi(2021, 2024)

# Note that 2023/2024 cpi data is subject to revisions
cpi = pd.concat([cpi_1, cpi_2])
cpi_sub = cpi[['year', 'value']]
cpi_sub.loc[:, 'value'] = pd.to_numeric(cpi_sub['value'])
cpi_grouped = cpi_sub.groupby('year')['value'].mean().reset_index()
cpi_grouped['inflation_perc_yoy'] = cpi_grouped['value'].pct_change() * 100

# Joining housing data with inflation data
grouped_year = grouped_year.merge(cpi_grouped, how = 'left', on = 'year')
grouped_year_state = grouped_year_state.merge(cpi_grouped, how = 'left', on = 'year')

# Function for getting inflation
def get_inflation(current):
    base = cpi_grouped['value'].iloc[0]
    if current == base:
        return np.nan
    inflation = round((current - base) / base * 100, 2)
    return inflation

In [8]:
grouped_year['cumulative_inflation_since_2012'] = grouped_year['value'].map(get_inflation)
grouped_year_state['cumulative_inflation_since_2012'] = grouped_year_state['value'].map(get_inflation)

In [9]:
# Use cumulative inflation to get median sale price adjusted by 2012 base year
def get_price_2012(df, input_col, output_col):
    df[output_col] = None
    for index, row in df.iterrows():
        price = row[input_col]
        inflation = row['cumulative_inflation_since_2012']
        if pd.notna(inflation):
            df.at[index, output_col] = int(round(price / (1 + inflation / 100), 0))
            
    return df

In [10]:
grouped_year = get_price_2012(grouped_year, 'median_sale_price', 'price_2012_adj')
grouped_year_state = get_price_2012(grouped_year_state,'median_sale_price', 'price_2012_adj')

In [11]:
# Joining housing sales + inflation with wage/salary data
wages_grouped = wages[['YEAR', 'A_MEDIAN']].groupby('YEAR').median().astype('Int64')

grouped_year = grouped_year.merge(wages_grouped, left_on = 'year', right_on = 'YEAR')
grouped_year.rename(columns = {'A_MEDIAN': 'Median_Salary'}, inplace = True)

# Wage data only goes up to 2023 so analysis can only reliably span 2012 - 2023
grouped_year_state = grouped_year_state.merge(wages, how = 'inner', left_on = ['year', 'state'], right_on = ['YEAR', 'STATE'])
grouped_year_state.drop(columns = ['OCC_CODE', 'AREA', 'YEAR', 'STATE'])
grouped_year_state.rename(columns = {'A_MEDIAN': 'Median_Salary'}, inplace = True)
grouped_year_state['median_sale_price'] = grouped_year_state['median_sale_price'].astype('Int64')

In [12]:
# Can now adjust wages for 2012 inflation
grouped_year = get_price_2012(grouped_year, 'Median_Salary', 'wage_2012_adj')
grouped_year_state = get_price_2012(grouped_year_state, 'Median_Salary', 'wage_2012_adj')

# Final Cleaning before visualizations
def cleaner(df):
    df.rename(columns = {'price_2012_adj': 'median_sale_price_adj', 'Median_Salary': 'median_salary', 'wage_2012_adj': 'median_salary_adj'}, inplace = True)
    return df

grouped_year = cleaner(grouped_year)
grouped_year_state = cleaner(grouped_year_state)

# Dropping duplicate cols
grouped_year_state.drop(columns = ['YEAR', 'STATE'], inplace = True)

## Streamlit Web App

Public URL: https://app-app-housing-gqe7gkq7dya3pgz8uwrbvr.streamlit.app/